In [16]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import time
import matplotlib.pyplot as plt
from scipy.stats import gamma
import math

$$\phi(x;\delta) = exp(-x^{1/\delta})$$

$$k(x;a) = exp(-ax)$$

In [39]:
def density(x, delta):
    return np.exp(- x ** (1 / delta))

def kernel(x, a):
    return np.exp(- a * x)

def CRN(N, a, seed = 123):
    np.random.seed(seed)
    u = np.random.uniform(low = 1e-8, high = 1,size = N)
    x =  - np.log(1 - u) / a
    return x

def get_weight(data_density, data_proposal):
    return data_density / data_proposal


def EIS_diagnostic(N, a, c, delta, seed = 123):
    np.random.seed(seed)
    Var = []
    for param in [a, 5 * a]:
        x = CRN(N, param)
        x_density = density(x, delta)
        x_kernel = kernel(x, param)
        x_proposal = kernel(x, param) * param
        weights = get_weight(x_density, x_proposal)
        Var.append(np.mean((((x_density / x_kernel) * np.exp(-c) + (x_kernel / x_density) * np.exp(c))- 2) * weights))
    return Var[1]/Var[0]


def EIS(N, delta, epsilon = 0.01, max_iter = 5000, seed = 123):
    a = 1/delta
    step = 0
    for iter in range(max_iter):
        # x = CRN(N, a, int(time.time()/(iter+2)))
        x = CRN(N, a, seed)
        # y = np.log(density(x, delta))
        # X = np.log(kernel(x, a))
        y = - x ** (1 / delta)
        X = - x
        y = np.reshape(y, (-1, 1))
        X = np.reshape(X, (-1, 1))
        lm_model = LinearRegression()
        lm_model.fit(X, y)
        a_new = lm_model.coef_[0][0]
        c = lm_model.intercept_[0]
        # X = sm.add_constant(X)
        # ols_model = sm.OLS(y, X).fit()
        # c = ols_model.params[0]
        # a_new = - ols_model.params[1]

        step += 1
        if np.abs(a_new - a) < epsilon:
            break
        else:
            a = a_new
    return a_new, c, step

def g_expectation(N, a, delta, seed = 123, moment = 0):
    np.random.seed(seed)
    x = np.random.exponential(scale = 1 / a, size= N)
    x_density = density(x, delta)
    x_kernel = kernel(x, a)
    x_proposal = x_kernel * a 
    weights = get_weight(x_density, x_proposal)
    return np.mean(x ** moment * weights)


In [40]:
deltas = [0.6, 0.8, 1.2, 1.6, 2.0, 2.4]
replications = 100
S = 100
EIS_estimators = {'EIS' : [], 'Sd': []}
EIS_iterations = {'Mean' : []}
EIS_var = {'Mean' : [], 'Sd' : []}
EIS_g0 = {'Mean' : [], 'Sd' : []}
EIS_g2 = {'Mean' : [], 'Sd' : []}
EIS_E2 = {'Mean' : [], 'Sd' : []}
for i in range(len(deltas)):
    estimators = []
    iters = []
    vars = []
    G0 = []
    G2 = []
    E2 = []
    for step in range(replications):
        estimator, c, iter = EIS(S, deltas[i], epsilon= 1e-2, seed = step)
        var = EIS_diagnostic( S, estimator, c, deltas[i], seed = step)
        g0 = g_expectation(S, estimator, deltas[i], seed=step)
        g2 = g_expectation(S, estimator, deltas[i], seed=step, moment = 2)
        estimators.append(estimator)
        iters.append(iter)
        vars.append(var)
        G0.append(g0)
        G2.append(g2)
        E2.append(g2/g0)
        
    EIS_estimators['EIS'].append(np.mean(estimators))
    EIS_estimators['Sd'].append(np.std(estimators))
    EIS_iterations['Mean'].append(np.mean(iters))
    EIS_var['Mean'].append(np.mean(vars))
    EIS_var['Sd'].append(np.std(vars))
    EIS_g0['Mean'].append(np.mean(G0))
    EIS_g0['Sd'].append(np.std(G0))
    EIS_g2['Mean'].append(np.mean(G2))
    EIS_g2['Sd'].append(np.std(G2))
    EIS_E2['Mean'].append(np.mean(E2))
    EIS_E2['Sd'].append(np.std(E2))

    



In [41]:
pd.DataFrame(EIS_estimators)

,EIS,Sd
0,1.691045,0.128468
1,1.306401,0.046664
2,0.755434,0.025497
3,0.412177,0.039442
4,0.213345,0.033393
5,0.109363,0.022784


In [42]:
pd.DataFrame(EIS_iterations)

,Mean
0,7.73
1,2.74
2,2.55
3,3.98
4,4.98
5,5.00


In [43]:
pd.DataFrame(EIS_g0)

,Mean,Sd
0,0.890290,0.024509
1,0.930959,0.011792
2,1.099987,0.012751
3,1.416109,0.046451
4,1.956294,0.102683
5,2.863786,0.209989


In [44]:
pd.DataFrame(EIS_g2)

,Mean,Sd
0,0.559895,0.078053
1,0.995478,0.148673
2,4.194829,1.274829
3,22.929817,12.967591
4,151.134975,129.984924
5,1126.603077,1357.892440


In [45]:
pd.DataFrame(EIS_E2)

,Mean,Sd
0,0.628484,0.082905
1,1.069778,0.161754
2,3.805215,1.132260
3,16.003807,8.714790
4,75.189080,61.704855
5,373.557069,420.107333


In [46]:
pd.DataFrame(EIS_var)

,Mean,Sd
0,370.910044,59.668910
1,1462.824271,126.698326
2,1406.564909,217.830690
3,162.603290,30.603200
4,63.180885,12.801749
5,34.448343,7.742814


In [16]:
def density2(x, delta):
    return ((1 + x ** 2 / (delta - 2)) ** ((- 1 / 2) * (delta + 1)))

def kernel2(x, a):
    return np.exp(- (a / 2) * (x ** 2))

def CRN2(N, a, seed = 123):
    np.random.seed(seed)
    u = np.random.normal(loc = 0, scale = 1,size = N)
    x =  0 + u / np.sqrt(a)
    return x

def get_weight(data_density, data_proposal):
    return data_density / data_proposal


def EIS_diagnostic2(N, a, c, delta, seed = 123):
    Var = []
    for param in [a, 5 * a]:
        x = CRN2(N, a, seed)
        x_density = density2(x, delta)
        x_kernel = kernel2(x, param)
        x_proposal = kernel2(x, a) * a
        weights = get_weight(x_density, x_proposal)
        Var.append(np.mean((((x_density / x_kernel) * np.exp(-c) + (x_kernel / x_density) * np.exp(c))- 2) * weights))
    return Var[1]/Var[0]


def EIS2(N, delta, epsilon = 0.01, max_iter = 5000, seed = 123):
    a = 1/delta
    step = 0
    for iter in range(max_iter):
        # x = CRN(N, a, int(time.time()/(iter+2)))
        x = CRN2(N, a, seed)
        y = (- (delta + 1) / 2 ) * np.log(1 + (x ** 2 / (delta - 2)))
        X = - (x ** 2) / 2
        y = np.reshape(y, (-1, 1))
        X = np.reshape(X, (-1, 1))
        lm_model = LinearRegression()
        lm_model.fit(X, y)
        a_new = lm_model.coef_[0][0]
        c = lm_model.intercept_[0]
        # X = sm.add_constant(X)
        # ols_model = sm.OLS(y, X).fit()
        # c = ols_model.params[0]
        # a_new = - ols_model.params[1]

        step += 1
        if np.abs(a_new - a) < epsilon:
            break
        else:
            a = a_new
    return a_new, c, step

def g_expectation2(N, a, delta, seed = 123, moment = 0):
    x = CRN2(N, a, seed)
    x_density = density2(x, delta)
    x_kernel = kernel2(x, a)
    x_proposal = x_kernel * np.sqrt(a / (2 * np.pi)) 
    weights = get_weight(x_density, x_proposal)
    return np.mean(x ** moment * weights)


In [17]:
deltas = [2.5, 3.0, 10.0, 150.0]
replications = 100
S = 100
EIS_estimators = {'EIS' : [], 'Sd': []}
EIS_iterations = {'Mean' : []}
EIS_var = {'Mean' : [], 'Sd' : []}
EIS_g0 = {'Mean' : [], 'Sd' : []}
EIS_g2 = {'Mean' : [], 'Sd' : []}
EIS_E2 = {'Mean' : [], 'Sd' : []}
for i in range(len(deltas)):
    estimators = []
    iters = []
    vars = []
    G0 = []
    G2 = []
    E2 = []
    for step in range(replications):
        estimator, c, iter = EIS2(S, deltas[i], epsilon= 1e-2, seed = step)
        var = EIS_diagnostic2( S, estimator, c, deltas[i], seed = step)
        g0 = g_expectation2(S, estimator, deltas[i], seed=step)
        g2 = g_expectation2(S, estimator, deltas[i], seed=step, moment = 2)

        estimators.append(estimator)
        iters.append(iter)
        vars.append(var)
        G0.append(g0)
        G2.append(g2)
        E2.append(g2/g0)
        
    EIS_estimators['EIS'].append(np.mean(estimators))
    EIS_estimators['Sd'].append(np.std(estimators))
    EIS_iterations['Mean'].append(np.mean(iters))
    EIS_var['Mean'].append(np.mean(vars))
    EIS_var['Sd'].append(np.std(vars))
    EIS_g0['Mean'].append(np.mean(G0))
    EIS_g0['Sd'].append(np.std(G0))
    EIS_g2['Mean'].append(np.mean(G2))
    EIS_g2['Sd'].append(np.std(G2))
    EIS_E2['Mean'].append(np.mean(E2))
    EIS_E2['Sd'].append(np.std(E2))

    



In [18]:
pd.DataFrame(EIS_estimators)

,EIS,Sd
0,3.210730,0.664087
1,2.033311,0.365848
2,1.081274,0.071424
3,1.002654,0.005021


In [19]:
pd.DataFrame(EIS_iterations)

,Mean
0,10.06
1,8.44
2,5.33
3,3.99


In [20]:
pd.DataFrame(EIS_g0)

,Mean,Sd
0,1.190403,0.051396
1,1.526251,0.055958
2,2.286711,0.026583
3,2.493411,0.001966


In [21]:
pd.DataFrame(EIS_g2)

,Mean,Sd
0,0.454271,0.283397
1,0.877267,0.450268
2,2.169480,0.445573
3,2.493369,0.352868


In [22]:
pd.DataFrame(EIS_E2)

,Mean,Sd
0,0.374527,0.206551
1,0.567232,0.260665
2,0.947476,0.187359
3,0.999959,0.141363
